In [1]:
import platform 
print(platform.architecture())


('64bit', 'WindowsPE')


In [2]:
#알수없음
from ast import keyword
# from asyncio import FastChildWatcher
from atexit import register
from glob import glob

#DB모듈
from sqlite3 import connect
from tkinter.ttk import Progressbar
from django import get_version
import pymysql
import sqlite3
# from test import fetcher
# pyrcc5 test.qrc -o ./test_rc.py

#시스템모듈
import sys
import os
import time
import datetime
import json

#UI모듈
from PyQt5.QtGui import *
from PyQt5 import uic
# from PyQt5.QtWidgets import *
from PyQt5.QtWidgets import QDialog, QMainWindow, QApplication, QMessageBox, QFileDialog, QTableWidgetItem
from PyQt5 import QtGui
# from PyQt5 import QtCore, QtGui, QtWidgets
import qdarktheme
from PyQt5.QtCore import Qt, QDate, QCoreApplication, QThread, QTimer
# from PyQt5.QtCore import *
#pip install pyqtdarktheme

#부가기능
import webbrowser
import requests
import threading
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
import re, uuid #MAC
import pickle
import csv
import gc
import random

#데이터분석
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score

#Misc
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import scale, MinMaxScaler

#자연어
import collections
from konlpy.tag import Okt
from collections import Counter

c:\Users\ekam3\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.0.0)/charset_normalizer (2.0.10) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [3]:
from math import gamma
import xgboost


ai_date = datetime.datetime.now()
ai_date = ai_date.date()
ai_date_n = ['%04d' % ai_date.year, '%02d' % ai_date.month,'%02d' % ai_date.day]
ai_date_n = ''.join(ai_date_n)
# print(ai_date_n)
ai_date = ai_date - datetime.timedelta(days=30)
ai_date = ['%04d' % ai_date.year, '%02d' % ai_date.month,'%02d' % ai_date.day]
ai_date_p = ''.join(ai_date)
# print(ai_date_p)

#카테고리추천
item_result = pd.read_excel('./aaa.xlsx', engine='openpyxl')
#중복카테고리 다 제거한 데이터프레임 생성
item_category = item_result.drop_duplicates(['카테고리'], ignore_index=True)
item_category = item_category[['No','대분류','중분류','소분류','세분류','카테고리']]
item_category.insert(6,'구매상품수_1개월','-')
item_category.insert(7,'등록상품수_1개월','-')
item_category.insert(8,'구매/등록','-')
item_category.insert(9,'경쟁강도_AI','-')
item_category.insert(10,'상품명_추천','-')
item_category.insert(11,'태그명_추천','-')
# print(item_category)

#원핫인코딩
item_result.loc[item_result['구매'] == 0, '구매'] = 0
item_result.loc[item_result['구매'] != 0, '구매'] = 1
item_result.loc[item_result['찜'] == 0, '찜'] = 0
item_result.loc[item_result['찜'] != 0, '찜'] = 1
item_result.loc[item_result['문의'] == 0, '문의'] = 0
item_result.loc[item_result['문의'] != 0, '문의'] = 1
item_result.loc[item_result['옵션_L'] == '', '옵션_L'] = 0
item_result.loc[item_result['옵션_L'] != '', '옵션_L'] = 1
item_result.loc[item_result['태그_L'] == '', '태그_L'] = 0
item_result.loc[item_result['태그_L'] != '', '태그_L'] = 1
item_result = item_result[['카테고리','순위','등록일자','구매','찜','문의','상품명','옵션_L','태그','태그_L']]
item_result = item_result.astype({'카테고리':'int','순위':'int','등록일자':'int','구매':'int','찜':'int','문의':'int','옵션_L':'int','태그_L':'int'})
# item_result = item_result.astype('int')
# print(item_result)

#테스트 데이터프레임 생성
item_predict = pd.DataFrame({'순위':[1000],
                             '등록일자':[ai_date_n],
                             '구매':[0],
                             '찜':[0],
                             '문의':[0],
                             '옵션_L':[0],
                             '태그_L':[0]})
# print(item_predict)
ac_category = list(item_category['카테고리'])

for idx, category in enumerate(ac_category):
    ac_category = (item_result.카테고리 == category)
    ac_date = (item_result.등록일자 > int(ai_date_p))
    ac_df = item_result.loc[ac_category & ac_date, ['순위', '등록일자', '구매', '찜', '문의', '옵션_L', '태그_L']]
    ac_pur = sum(list(ac_df['구매']))
    ac_app = len(list(ac_df['구매']))

    #머신러닝
    item_predict.loc[0, '등록일자'] = int(ac_df['등록일자'].max())
    item_predict.loc[0, '순위'] = int(ac_df['순위'].max())

    x_train = ac_df.drop(['구매'], axis=1)
    y_train = ac_df['구매']
    x_test = item_predict.drop(['구매'], axis=1)
    y_test = item_predict['구매']

    scaler = MinMaxScaler()
    scaler.fit(x_train)
    scaler.fit(x_test)

    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

    kf = KFold(n_splits=10, random_state=1, shuffle=True)

    def rmsle(y_test, y_pred):
        return np.sqrt(mean_squared_error(y_test, y_pred))
    def cv_rmse(model, x_train=x_train):
        rmse = np.sqrt(-cross_val_score(model, x_train, y_train, scoings='meg_mean_squared_error', cv=kf))

    xgboost = XGBRegressor(learning_rate=0.3,
                            n_estimators=500,
                            max_depth=3,
                            min_child_weight=0,
                            gamma=0.6,
                            subsample=0.7,
                            colsample_bytree=0.7,
                            objective='reg:linear',
                            nthread=-1,
                            scale_pos_weight=1,
                            seed=27,
                            reg_alpha=0.00006,
                            random_state=42)
    xgb_model_full_data = xgboost.fit(x_train, y_train)
    ac_result = xgb_model_full_data.predict(x_test)

    item_category.loc[idx, '구매상품수_1개월'] = ac_pur
    item_category.loc[idx, '등록상품수_1개월'] = ac_app
    item_category.loc[idx, '구매/등록'] = '%0.6f' % float(ac_pur/ac_app)
    item_category.loc[idx, '경쟁강도_AI'] = ac_result

# print(item_category)


[16:03:17] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[16:03:17] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[16:03:18] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[16:03:18] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[16:03:18] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[16:03:18] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:20

In [49]:
#############################
#상품추천
ai_date = datetime.datetime.now()
ai_date = ai_date.date()
ai_date_nn = ['%04d' % ai_date.year, '%02d' % ai_date.month,'%02d' % ai_date.day]
ai_date_nn = ''.join(ai_date_n)
# print(ai_date_n)
ai_date = ai_date - datetime.timedelta(days=90)
ai_date = ['%04d' % ai_date.year, '%02d' % ai_date.month,'%02d' % ai_date.day]
ai_date_pp = ''.join(ai_date)
# print(ai_date_pp)
ac_category = list(item_category['카테고리'])

for idx, category in enumerate(ac_category):
    item_result = item_result.astype({'등록일자':'int'})
    an_category = (item_result.카테고리 == category)
    an_date = (item_result.등록일자 > int(ai_date_pp))
    an_pur = (item_result.구매 != 0)
    an_zzim = (item_result.찜 != 0)
    an_con = (item_result.문의 != 0)
    an_df = item_result.loc[an_category & an_date & (an_pur | an_zzim | an_con), ['상품명', '태그']]
    # print(an_df)
    an_title = an_df['상품명'].tolist()
    an_title = ' '.join(map(str, an_title))
    an_tag = an_df['태그']
    an_tag = an_tag.dropna(axis=0)
    an_tag = an_tag.tolist()

    an_tag = ' '.join(map(str, an_tag))
    an_title = an_title.split(' ')
    an_tag = an_tag.split(',')

    an_title_list = collections.Counter(an_title)
    an_tag_list = collections.Counter(an_tag)
    an_title_most = an_title_list.most_common(10)
    an_tag_most = an_tag_list.most_common(10)
    an_title_list = {}
    an_tag_list = {}
    for n, c in an_title_most:
        an_title_list[n] = c
    for n, c in an_tag_most:
        an_tag_list[n] = c
    an_title_list = ' '.join(map(str, an_title_list.keys()))
    an_tag_list = ' '.join(map(str, an_tag_list.keys()))
    
    item_category.loc[idx, '상품명_추천'] = an_title_list
    item_category.loc[idx, '상품명_추천'] = an_tag_list

print(item_category)

    No      대분류       중분류    소분류          세분류      카테고리 구매상품수_1개월 등록상품수_1개월  \
0    1  가구/인테리어  DIY자재/용품  가구부속품         가구다리  50003307         0        34   
1    2  가구/인테리어  DIY자재/용품  가구부속품         가구바퀴  50003308         0        64   
2    3  가구/인테리어  DIY자재/용품  가구부속품  경첩/꺽쇠/자석철물류  50003309         1        33   
3    4  가구/인테리어  DIY자재/용품  가구부속품      기타가구부속품  50003314         5        10   
4    5  가구/인테리어  DIY자재/용품  가구부속품         나사/못  50003310         0        15   
5    6  가구/인테리어  DIY자재/용품  가구부속품        로프/철망  50003311         0        41   
6    8  가구/인테리어  DIY자재/용품  가구부속품      소켓/옷걸이봉  50003313         0        62   
7    9  가구/인테리어  DIY자재/용품  데코스티커          NaN  50001066         5       693   
8   10  가구/인테리어  DIY자재/용품   리모델링          싱크대  50008228         2        41   
9   11  가구/인테리어  DIY자재/용품   리모델링        욕실/바스  50008208         0        29   
10  12  가구/인테리어  DIY자재/용품   리모델링           중문  50008188         0         3   

       구매/등록   경쟁강도_AI                             

In [79]:
import pymysql
recommend_code = 'youngmusk0308'
db_id = 'musk'
con_user = pymysql.connect(host='3.39.22.73', user='young_write', password='0000', db='trend', charset='utf8')
cur_user = con_user.cursor()
#프로그램 버젼 확인
sql = "UPDATE recommend SET cnt=cnt+1 WHERE code=%s;"
cur_user.execute(sql, recommend_code)
sql = "UPDATE check_login SET recommend=recommend+1 WHERE id=%s;"
cur_user.execute(sql, db_id)
con_user.commit()
con_user.close()

In [72]:
a = 'https://living-only-today.tistory.com/82'
print(a)

a != ''

https://living-only-today.tistory.com/82


True

In [54]:
import datetime
dt_now = datetime.datetime.now()
register_date = dt_now.date()
dt_pre = dt_now + datetime.timedelta(days=3)
premium_date = dt_pre.date()

a = register_date - premium_date
print(a.days)

-3


In [80]:
import pymysql
id = 'musk'
try:
    con_user = pymysql.connect(host='3.39.22.73', user='youngmusk', password='9404', db='trend', charset='utf8')
    cur_user = con_user.cursor()
    #프리미엄 기간 추가
    sql = "UPDATE check_login SET premium=DATE_ADD(premium, interval 3 DAY) WHERE id=%s;"
    # sql = "UPDATE check_login SET premium=DATE_SUB(premium, interval 3 DAY) WHERE id=%s;"
    cur_user.execute(sql, id)
    sql = "UPDATE youngmusk_login SET premium=DATE_ADD(premium, interval 3 DAY) WHERE id=%s;"
    # sql = "UPDATE youngmusk_login SET premium=DATE_SUB(premium, interval 3 DAY) WHERE id=%s;"
    cur_user.execute(sql, id)
except:
    print(1)
finally:
    con_user.commit()
    con_user.close()

In [33]:
import pymysql
d = 'musk'
try:
    con_user = pymysql.connect(host='3.39.22.73', user='young_write', password='0000', db='trend', charset='utf8')
    cur_user = con_user.cursor()
    #프로그램 버젼 확인
    sql = "UPDATE check_login SET review=review+1 WHERE id=%s;"
    cur_user.execute(sql, d)
except:
    print(1)
finally:
    con_user.commit()
    con_user.close()

In [24]:
import pymysql
ddd= 0

con_user = pymysql.connect(host='3.39.22.73', user='young_write', password='0000', db='trend', charset='utf8')
cur_user = con_user.cursor()
sql = f"INSERT INTO review_list (id, link, give, date) VALUES ('222','222','1','2022-01-01');"
cur_user.execute(sql)

print("ddd")

con_user.commit()
con_user.close()

ddd


In [14]:
import pymysql
try:
    con_user = pymysql.connect(host='3.39.22.73', user='young_read', password='0000', db='trend', charset='utf8')
    cur_user = con_user.cursor()
    sql = "SELECT * FROM web_link WHERE button=%s;"
    cur_user.execute(sql, '7일 무료체험')
    free_seven = cur_user.fetchall()
    free_seven_link = free_seven[0][2]
    print(free_seven_link)
except:
    # msg.setWindowTitle('알림')
    # msg.setText('신규 회원가입을 진행해주시거나\n회원가입을 진행했던 PC에서 로그인을 시도해주세요.')
    # msg.exec_()
    print("실패")
    # return

https://youngmusk.imweb.me/TREND/?q=YToxOntzOjEyOiJrZXl3b3JkX3R5cGUiO3M6MzoiYWxsIjt9&bmode=view&idx=13117956&t=board
